In [1]:
import numpy as np
import os
import random
from os.path import join
from keras import layers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, LSTM, Bidirectional, Flatten
from keras.utils import to_categorical
from keras_utils import set_keras_session
from inception_generators import frames_generator_rnn,load_whole_dataset
from plot_utils import plot_history

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
set_keras_session()
dataset = 'UCF11'
nb_classes = int(dataset[3:])

In [3]:
frame_number = 50
width = 320
height = 240
channels = 3
padding = None

dataset_name= 'separate_frames_{}_h_{}_w_{}_inception'.format(frame_number, height, width)
if padding is not None:
    dataset_name += '_padding_{}'.format(padding)


dataset_dir = join('datasets', dataset, dataset_name)
(X_train, Y_train), (X_valid, Y_valid) = list(load_whole_dataset(dataset_dir, 'rnn', ['train', 'valid']))
sequence_shape = (frame_number, height, width, channels)
print(X_train.shape)

(1126, 50, 2048)


In [4]:
from keras.applications.inception_v3 import InceptionV3
from keras.applications.densenet import DenseNet121
from keras.models import Model
from keras import backend as K
from keras import regularizers

def build_model():
       
    rnn = Sequential()
    rnn.add(Dropout(0.5, input_shape=(X_train.shape[1:])))
    rnn.add(LSTM(512, return_sequences=True, input_shape=(X_train.shape[1:]), dropout=0.5, kernel_regularizer=regularizers.L1L2(l1=0.0, l2=0.01)))
    #rnn.add(Dense(512, activation='relu'))
    rnn.add(Flatten())
    rnn.add(Dense(nb_classes, activation='softmax'))

    return rnn


def build_model_stacked():
       
    rnn = Sequential()
    rnn.add(LSTM(512, input_shape=(X_train.shape[1:]), recurrent_dropout=0.2, return_sequences=True))
    rnn.add(LSTM(128, input_shape=(X_train.shape[1:])))
    #rnn.add(Dense(128, activation='relu'))
    rnn.add(Dropout(0.5))
    rnn.add(Dense(nb_classes, activation='softmax'))

    return rnn

In [9]:
from keras import metrics, callbacks, optimizers
from functools import partial

top_3_k_categorical_accuracy = partial(metrics.top_k_categorical_accuracy, k=3)
top_3_k_categorical_accuracy.__name__ = 'top_3'

early_stopper = callbacks.EarlyStopping(patience=5)
reduce_lr = callbacks.ReduceLROnPlateau(patience=5, factor=0.75)

model = build_model()
print(model.summary())

sgd = optimizers.SGD(momentum=0.9, nesterov=True, lr=0.001)
sgd = optimizers.SGD(momentum=0.9, lr=0.006, clipnorm=2.)
adam = optimizers.Adam(lr=0.0001, clipnorm=1.)

model.compile(optimizer=sgd, loss='categorical_crossentropy', 
              metrics=['accuracy', top_3_k_categorical_accuracy])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout_3 (Dropout)          (None, 50, 2048)          0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 50, 512)           5244928   
_________________________________________________________________
flatten_3 (Flatten)          (None, 25600)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 11)                281611    
Total params: 5,526,539
Trainable params: 5,526,539
Non-trainable params: 0
_________________________________________________________________
None


In [10]:
history = model.fit(X_train, Y_train, validation_data=(X_valid, Y_valid), 
          batch_size=64, epochs=500, callbacks=[reduce_lr])

Train on 1126 samples, validate on 469 samples
Epoch 1/500
1126/1126 [==============================] - 4s 3ms/step - loss: 22.5546 - acc: 0.3206 - top_3: 0.5302 - val_loss: 21.5361 - val_acc: 0.6823 - val_top_3: 0.9147
Epoch 2/500
1126/1126 [==============================] - 2s 2ms/step - loss: 21.3306 - acc: 0.7220 - top_3: 0.9112 - val_loss: 20.8174 - val_acc: 0.7420 - val_top_3: 0.9232
Epoch 3/500
1126/1126 [==============================] - 2s 2ms/step - loss: 20.5708 - acc: 0.8268 - top_3: 0.9689 - val_loss: 20.4051 - val_acc: 0.7846 - val_top_3: 0.9318
Epoch 4/500
1126/1126 [==============================] - 2s 2ms/step - loss: 20.0503 - acc: 0.8783 - top_3: 0.9876 - val_loss: 19.9856 - val_acc: 0.8273 - val_top_3: 0.9510
Epoch 5/500
1126/1126 [==============================] - 2s 2ms/step - loss: 19.6097 - acc: 0.9023 - top_3: 0.9920 - val_loss: 19.5490 - val_acc: 0.8443 - val_top_3: 0.9723
Epoch 6/500
1126/1126 [==============================] - 2s 2ms/step - loss: 19.1747 - a

1126/1126 [==============================] - 3s 2ms/step - loss: 5.5973 - acc: 0.9849 - top_3: 1.0000 - val_loss: 6.1420 - val_acc: 0.8742 - val_top_3: 0.9787
Epoch 49/500
1126/1126 [==============================] - 2s 2ms/step - loss: 5.4150 - acc: 0.9920 - top_3: 1.0000 - val_loss: 6.0683 - val_acc: 0.8635 - val_top_3: 0.9744
Epoch 50/500
1126/1126 [==============================] - 2s 2ms/step - loss: 5.2448 - acc: 0.9858 - top_3: 1.0000 - val_loss: 5.7082 - val_acc: 0.8678 - val_top_3: 0.9851
Epoch 51/500
1126/1126 [==============================] - 2s 2ms/step - loss: 5.0743 - acc: 0.9876 - top_3: 0.9991 - val_loss: 5.4370 - val_acc: 0.8806 - val_top_3: 0.9851
Epoch 52/500
1126/1126 [==============================] - 2s 2ms/step - loss: 4.9092 - acc: 0.9938 - top_3: 0.9991 - val_loss: 5.6104 - val_acc: 0.8721 - val_top_3: 0.9616
Epoch 53/500
1126/1126 [==============================] - 2s 2ms/step - loss: 4.7348 - acc: 0.9947 - top_3: 1.0000 - val_loss: 5.1455 - val_acc: 0.8827 -

Epoch 96/500
1126/1126 [==============================] - 3s 2ms/step - loss: 1.2404 - acc: 0.9902 - top_3: 1.0000 - val_loss: 1.6680 - val_acc: 0.8891 - val_top_3: 0.9915
Epoch 97/500
1126/1126 [==============================] - 3s 2ms/step - loss: 1.2008 - acc: 0.9902 - top_3: 1.0000 - val_loss: 1.6843 - val_acc: 0.8955 - val_top_3: 0.9765
Epoch 98/500
1126/1126 [==============================] - 3s 2ms/step - loss: 1.1716 - acc: 0.9911 - top_3: 1.0000 - val_loss: 1.6929 - val_acc: 0.8849 - val_top_3: 0.9787
Epoch 99/500
1126/1126 [==============================] - 3s 2ms/step - loss: 1.1291 - acc: 0.9929 - top_3: 1.0000 - val_loss: 1.6275 - val_acc: 0.8913 - val_top_3: 0.9808
Epoch 100/500
1126/1126 [==============================] - 3s 2ms/step - loss: 1.0918 - acc: 0.9938 - top_3: 1.0000 - val_loss: 1.7350 - val_acc: 0.8678 - val_top_3: 0.9616
Epoch 101/500
1126/1126 [==============================] - 3s 2ms/step - loss: 1.0696 - acc: 0.9920 - top_3: 1.0000 - val_loss: 1.5909 - va

1126/1126 [==============================] - 3s 2ms/step - loss: 0.3462 - acc: 0.9956 - top_3: 1.0000 - val_loss: 0.9362 - val_acc: 0.8849 - val_top_3: 0.9808
Epoch 144/500
1126/1126 [==============================] - 3s 2ms/step - loss: 0.3359 - acc: 0.9964 - top_3: 1.0000 - val_loss: 0.8740 - val_acc: 0.8955 - val_top_3: 0.9787
Epoch 145/500
1126/1126 [==============================] - 3s 2ms/step - loss: 0.3287 - acc: 0.9982 - top_3: 1.0000 - val_loss: 0.9063 - val_acc: 0.8849 - val_top_3: 0.9829
Epoch 146/500
1126/1126 [==============================] - 3s 2ms/step - loss: 0.3285 - acc: 0.9964 - top_3: 1.0000 - val_loss: 0.8817 - val_acc: 0.8870 - val_top_3: 0.9808
Epoch 147/500
1126/1126 [==============================] - 3s 2ms/step - loss: 0.3171 - acc: 1.0000 - top_3: 1.0000 - val_loss: 0.8636 - val_acc: 0.9019 - val_top_3: 0.9723
Epoch 148/500
1126/1126 [==============================] - 3s 2ms/step - loss: 0.3156 - acc: 0.9982 - top_3: 1.0000 - val_loss: 0.7895 - val_acc: 0.8

1126/1126 [==============================] - 2s 2ms/step - loss: 0.2314 - acc: 0.9973 - top_3: 1.0000 - val_loss: 0.7246 - val_acc: 0.8998 - val_top_3: 0.9893
Epoch 191/500
1126/1126 [==============================] - 2s 2ms/step - loss: 0.2261 - acc: 1.0000 - top_3: 1.0000 - val_loss: 0.7301 - val_acc: 0.8998 - val_top_3: 0.9808
Epoch 192/500
1126/1126 [==============================] - 2s 2ms/step - loss: 0.2263 - acc: 0.9982 - top_3: 1.0000 - val_loss: 0.7428 - val_acc: 0.9019 - val_top_3: 0.9808
Epoch 193/500
1126/1126 [==============================] - 2s 2ms/step - loss: 0.2244 - acc: 1.0000 - top_3: 1.0000 - val_loss: 0.7220 - val_acc: 0.9041 - val_top_3: 0.9808
Epoch 194/500
1126/1126 [==============================] - 2s 2ms/step - loss: 0.2258 - acc: 0.9991 - top_3: 1.0000 - val_loss: 0.6982 - val_acc: 0.9019 - val_top_3: 0.9872
Epoch 195/500
1126/1126 [==============================] - 2s 2ms/step - loss: 0.2251 - acc: 0.9982 - top_3: 1.0000 - val_loss: 0.7170 - val_acc: 0.8

...

1126/1126 [==============================] - 2s 2ms/step - loss: 0.2183 - acc: 0.9991 - top_3: 1.0000 - val_loss: 0.7015 - val_acc: 0.9019 - val_top_3: 0.9829
Epoch 285/500
1126/1126 [==============================] - 2s 2ms/step - loss: 0.2186 - acc: 0.9991 - top_3: 1.0000 - val_loss: 0.7017 - val_acc: 0.9019 - val_top_3: 0.9829
Epoch 286/500
1126/1126 [==============================] - 2s 2ms/step - loss: 0.2172 - acc: 1.0000 - top_3: 1.0000 - val_loss: 0.7016 - val_acc: 0.9019 - val_top_3: 0.9829
Epoch 287/500
1126/1126 [==============================] - 2s 2ms/step - loss: 0.2173 - acc: 0.9991 - top_3: 1.0000 - val_loss: 0.7016 - val_acc: 0.9019 - val_top_3: 0.9829
Epoch 288/500
1126/1126 [==============================] - 2s 2ms/step - loss: 0.2195 - acc: 0.9982 - top_3: 1.0000 - val_loss: 0.7015 - val_acc: 0.9019 - val_top_3: 0.9829
Epoch 289/500
1126/1126 [==============================] - 2s 2ms/step - loss: 0.2177 - acc: 0.9991 - top_3: 1.0000 - val_loss: 0.7015 - val_acc: 0.9

1126/1126 [==============================] - 2s 2ms/step - loss: 0.2169 - acc: 1.0000 - top_3: 1.0000 - val_loss: 0.7012 - val_acc: 0.9019 - val_top_3: 0.9829
Epoch 332/500
1126/1126 [==============================] - 2s 2ms/step - loss: 0.2166 - acc: 0.9991 - top_3: 1.0000 - val_loss: 0.7012 - val_acc: 0.9019 - val_top_3: 0.9829
Epoch 333/500
1126/1126 [==============================] - 2s 2ms/step - loss: 0.2162 - acc: 1.0000 - top_3: 1.0000 - val_loss: 0.7012 - val_acc: 0.9019 - val_top_3: 0.9829
Epoch 334/500
1126/1126 [==============================] - 2s 2ms/step - loss: 0.2154 - acc: 1.0000 - top_3: 1.0000 - val_loss: 0.7012 - val_acc: 0.9019 - val_top_3: 0.9829
Epoch 335/500
1126/1126 [==============================] - 2s 2ms/step - loss: 0.2171 - acc: 0.9991 - top_3: 1.0000 - val_loss: 0.7012 - val_acc: 0.9019 - val_top_3: 0.9829
Epoch 336/500
1126/1126 [==============================] - 2s 2ms/step - loss: 0.2169 - acc: 0.9991 - top_3: 1.0000 - val_loss: 0.7012 - val_acc: 0.9

1126/1126 [==============================] - 2s 2ms/step - loss: 0.2193 - acc: 0.9982 - top_3: 1.0000 - val_loss: 0.7012 - val_acc: 0.9019 - val_top_3: 0.9829
Epoch 379/500
1126/1126 [==============================] - 2s 2ms/step - loss: 0.2180 - acc: 0.9982 - top_3: 1.0000 - val_loss: 0.7012 - val_acc: 0.9019 - val_top_3: 0.9829
Epoch 380/500
1126/1126 [==============================] - 2s 2ms/step - loss: 0.2205 - acc: 0.9982 - top_3: 1.0000 - val_loss: 0.7012 - val_acc: 0.9019 - val_top_3: 0.9829
Epoch 381/500
1126/1126 [==============================] - 2s 2ms/step - loss: 0.2178 - acc: 0.9991 - top_3: 1.0000 - val_loss: 0.7012 - val_acc: 0.9019 - val_top_3: 0.9829
Epoch 382/500
1126/1126 [==============================] - 2s 2ms/step - loss: 0.2176 - acc: 0.9982 - top_3: 1.0000 - val_loss: 0.7012 - val_acc: 0.9019 - val_top_3: 0.9829
Epoch 383/500
1126/1126 [==============================] - 2s 2ms/step - loss: 0.2162 - acc: 1.0000 - top_3: 1.0000 - val_loss: 0.7012 - val_acc: 0.9

1126/1126 [==============================] - 2s 2ms/step - loss: 0.2155 - acc: 1.0000 - top_3: 1.0000 - val_loss: 0.7012 - val_acc: 0.9019 - val_top_3: 0.9829
Epoch 426/500
1126/1126 [==============================] - 2s 2ms/step - loss: 0.2169 - acc: 0.9991 - top_3: 1.0000 - val_loss: 0.7012 - val_acc: 0.9019 - val_top_3: 0.9829
Epoch 427/500
1126/1126 [==============================] - 2s 2ms/step - loss: 0.2155 - acc: 1.0000 - top_3: 1.0000 - val_loss: 0.7012 - val_acc: 0.9019 - val_top_3: 0.9829
Epoch 428/500
1126/1126 [==============================] - 2s 2ms/step - loss: 0.2177 - acc: 0.9991 - top_3: 1.0000 - val_loss: 0.7012 - val_acc: 0.9019 - val_top_3: 0.9829
Epoch 429/500
1126/1126 [==============================] - 2s 2ms/step - loss: 0.2162 - acc: 1.0000 - top_3: 1.0000 - val_loss: 0.7012 - val_acc: 0.9019 - val_top_3: 0.9829
Epoch 430/500
1126/1126 [==============================] - 2s 2ms/step - loss: 0.2186 - acc: 0.9982 - top_3: 1.0000 - val_loss: 0.7012 - val_acc: 0.9

1126/1126 [==============================] - 2s 2ms/step - loss: 0.2198 - acc: 0.9982 - top_3: 1.0000 - val_loss: 0.7012 - val_acc: 0.9019 - val_top_3: 0.9829
Epoch 473/500
1126/1126 [==============================] - 2s 2ms/step - loss: 0.2194 - acc: 0.9991 - top_3: 1.0000 - val_loss: 0.7012 - val_acc: 0.9019 - val_top_3: 0.9829
Epoch 474/500
1126/1126 [==============================] - 2s 2ms/step - loss: 0.2169 - acc: 0.9991 - top_3: 1.0000 - val_loss: 0.7012 - val_acc: 0.9019 - val_top_3: 0.9829
Epoch 475/500
1126/1126 [==============================] - 2s 2ms/step - loss: 0.2215 - acc: 0.9973 - top_3: 1.0000 - val_loss: 0.7012 - val_acc: 0.9019 - val_top_3: 0.9829
Epoch 476/500
1126/1126 [==============================] - 2s 2ms/step - loss: 0.2163 - acc: 1.0000 - top_3: 1.0000 - val_loss: 0.7012 - val_acc: 0.9019 - val_top_3: 0.9829
Epoch 477/500
1126/1126 [==============================] - 2s 2ms/step - loss: 0.2173 - acc: 1.0000 - top_3: 1.0000 - val_loss: 0.7012 - val_acc: 0.9

In [ ]:
plot_history(history)

In [ ]:
sgd = optimizers.SGD(momentum=0.9, lr=0.0005)

model.compile(optimizer=sgd, loss='categorical_crossentropy', 
              metrics=['accuracy', top_3_k_categorical_accuracy])

history = model.fit(X_train, Y_train, validation_data=(X_valid, Y_valid), 
          batch_size=64, epochs=50, callbacks=[reduce_lr])